In [ ]:
# plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# data analysis libraries
import pandas as pd
import numpy as np

# general imports
from pathlib import Path

### Load the data from the experiments

In [ ]:
folder = Path("storage/synced")

# load all csv files in the folder using pandas
df = pd.concat([pd.read_csv(f) for f in folder.glob("*.csv")])
df["stride_series"].replace(np.NaN, False, inplace=True)
df['train_exc_limit'].replace(np.NaN, 0, inplace=True)
df["pretrained"].replace(np.NaN, False, inplace=True)
df["pretrain_mode"].replace(np.NaN, False, inplace=True)
df["window_time_stride"].replace(np.NaN, 1, inplace=True)
df["window_patt_stride"].replace(np.NaN, 1, inplace=True)
df["cv_rep"].replace(np.NaN, False, inplace=True)
df["eq_wdw_length"] = df["window_length"]*df["window_time_stride"]
df

NameError: name 'Path' is not defined

In [11]:
df[df["arch"] == "nn"][['mode', 'arch', 'dataset', 'pretrain_mode', 'window_length', "stride_series",
                 'window_time_stride', 'window_patt_stride', 'train_exc_limit', 'pretrained', "cv_rep"]]

NameError: name 'df' is not defined

In [12]:
# check for duplicate entries
file_entries = len(df)
df = df.groupby(['mode', 'arch', 'dataset', 'pretrain_mode', 'window_length', "stride_series",
                 'window_time_stride', 'window_patt_stride', 'train_exc_limit', 'pretrained', "cv_rep"]).first().reset_index()
unique_entries = len(df)
print(f"{file_entries - unique_entries} duplicate entries removed")
# df = df.groupby(['mode', 'arch', 'dataset', 'pretrain_mode', 'window_length', "stride_series",
#                  'window_time_stride', 'window_patt_stride', 'train_exc_limit', 'pretrained', "cv_rep"]).count()
# df[df["eq_wdw_length"] > 1]

NameError: name 'df' is not defined

In [13]:
# show pretrained encoders
df[df["pretrain_mode"]==True][['mode', 'arch', 'dataset', 'pretrain_mode', 'window_length', 'window_time_stride', 'window_patt_stride', 'pretrained', "cv_rep"]]

NameError: name 'df' is not defined

In [14]:
df[np.logical_and(df["stride_series"]==False, df["pretrain_mode"]==False)]

NameError: name 'df' is not defined

In [15]:
cols = ['test_acc', 'test_f1', 'test_auroc', 'val_acc', 'val_f1', 'val_auroc']
df.columns

NameError: name 'df' is not defined

# Section 5.1: Time Dilation Effects

- Check the dependence of time dilation on classifier performance

In [16]:
# Filter the data
data = df[df["pretrain_mode"] == False].copy()

data = data[data['train_exc_limit'] == 32]
data = data[data["window_patt_stride"] == 1]
data = data[data["pretrained"] == False]

nn_data = data[data["arch"] == "nn"]
data = data[~data["arch"].isin(["nn"])]
#data = data[data["dataset"] == "CBF"]


NameError: name 'df' is not defined

In [17]:
# Generate a table
data.groupby(["dataset", "mode", "arch", "eq_wdw_length"])[cols].mean()

NameError: name 'data' is not defined

In [18]:
nn_data.groupby(["dataset", "mode", "arch"])[cols].mean()

NameError: name 'nn_data' is not defined

In [19]:
# Generate a plot
data["Method"] =  data["arch"] + "_" + data["mode"]
data.sort_values(["Method"], inplace=True)

sns.set_theme()
sns.set_style("whitegrid")
plt.rc('font', family='serif')
plt.rc('xtick', labelsize='small')
plt.rc('ytick', labelsize='small')

g = sns.relplot(data=data, x="eq_wdw_length", y="test_acc", hue='arch', kind="line", palette="bright", col="mode", row="dataset",
                height=6, aspect=1.5, legend=True, markers="True", facet_kws={"despine": False})

g.set_ylabels("Test Accuracy (%)")
g.set_xlabels("Equivalent Window Length");

g.savefig("figures/time_dilation.pdf", bbox_inches='tight')

NameError: name 'data' is not defined

# Section 5.2: Pattern Stride Effects

In [20]:
# data = df[df["pretrain_mode"] == False].copy()

# data = data[data['train_exc_limit'] == 32]
# data = data[data["window_time_stride"] == 7]
# data = data[data["pretrained"] == False]

# data = data[data["dataset"] == "CBF"]

In [21]:
# # Generate a table
# data.groupby(["dataset", "mode", "arch", "window_patt_stride"])[cols].mean()

In [22]:
# # Generate a plot
# data["Method"] =  data["arch"] + "_" + data["mode"]
# data.sort_values(["Method"], inplace=True)

# sns.set_theme()
# sns.set_style("whitegrid")
# plt.rc('font', family='serif')
# plt.rc('xtick', labelsize='small')
# plt.rc('ytick', labelsize='small')

# g = sns.relplot(data=data, x="window_patt_stride", y="val_acc", hue='Method', kind="line", palette="bright",
#                 height=8, aspect=1.5, legend=True, markers="True", facet_kws={"despine": False})

# g.set_ylabels("Test Accuracy (%)")
# g.set_xlabels("Window Pattern Stride");

# g.savefig("figures/pattern_stride_acc.pdf", bbox_inches='tight')

In [23]:
# # Generate a plot

# sns.set_theme()
# sns.set_style("whitegrid")
# plt.rc('font', family='serif')
# plt.rc('xtick', labelsize='small')
# plt.rc('ytick', labelsize='small')

# data["Time"] = data["time_train"]/60/60

# g = sns.relplot(data=data, x="window_patt_stride", y='Time', hue='Method', kind="line", palette="bright",
#                 height=8, aspect=1.5, legend=True, markers="True", facet_kws={"despine": False})

# g.set_ylabels("Training Time (Hours)")
# g.set_xlabels("Window Pattern Stride");

# g.savefig("figures/pattern_stride_time.pdf", bbox_inches='tight')

# Section 5.3: Self-Supervised Pretrain Effects

In [24]:
data = df[df["pretrain_mode"] == False].copy()

data = data[data["window_time_stride"] == 7]
data = data[data["window_patt_stride"] == 1]

# data = data[data["dataset"] == "CBF"]
# data = data[data["mode"] == "df"]
data = data[data["arch"].isin(["cnn", "res"])]

NameError: name 'df' is not defined

In [25]:
data[data["mode"] == "df"].groupby(["dataset", "mode", "arch", "pretrained", "stride_series", 'train_exc_limit'])[cols].mean()

NameError: name 'data' is not defined

In [26]:
# Generate a plot
data["Model"] = data["arch"] + " " + data["pretrained"].replace({True: "(Pret. ", False: ""}) + data["stride_series"].replace({True: "B)", False: "A)"})
data["Model"].replace({"cnn A)": "cnn", "res A)": "res", "tcn A)": "tcn"}, inplace=True)
data.sort_values(['Model'], inplace=True)

sns.set_theme()
sns.set_style("whitegrid")
plt.rc('font', family='serif')
plt.rc('xtick', labelsize='small')
plt.rc('ytick', labelsize='small')

g = sns.relplot(data=data, x="train_exc_limit", y="test_acc", hue='Model', kind="line", palette="bright", col="mode", row="dataset", 
                height=5, aspect=1.5, legend=True, facet_kws={"despine": False})

g.set_ylabels("Test Accuracy (%)")
g.set_xlabels("Samples per class");

g.savefig("figures/pretrain.pdf", bbox_inches='tight')

NameError: name 'data' is not defined

# heatmap showing percent change over no pretraied model

In [27]:
dfg = data.groupby(["dataset", "mode", "arch", "pretrained", "stride_series", 'train_exc_limit'])[cols].mean().reset_index()

metric = "test_acc"

# archs = {arch: i for i, arch in enumerate(dfg["arch"].sort_values().unique())}
# dsets = {dset: i for i, dset in enumerate(dfg["dataset"].sort_values().unique())}
# modes = {mode: i for i, mode in enumerate(dfg["mode"].sort_values().unique())}
# samps = {samp: i for i, samp in enumerate(dfg["train_exc_limit"].sort_values().unique())}
# prets = {"A": 0, "B": 1}

dfd = pd.DataFrame(columns=["arch", "mode", "dataset", "train_exc_limit", "pret", metric])

for (dset, mode, arch, nsamp), g in dfg.groupby(["dataset", "mode", "arch", 'train_exc_limit']):
    
    g: pd.DataFrame

    if len(g) == 3:

        def_val = g[metric].iloc[0]
        pa_val = g[metric].iloc[1]
        pb_val = g[metric].iloc[2]

        dfd.loc[len(dfd)] = [arch, mode, dset, nsamp, "A", (pa_val - def_val)*100]
        dfd.loc[len(dfd)] = [arch, mode, dset, nsamp, "B", (pb_val - def_val)*100]
        
    else: 
        continue

dfd["arch_pret"] = dfd["arch"] + "_" + dfd["pret"]
dfd.sort_values(["arch_pret", "dataset", "train_exc_limit"], inplace=True)

NameError: name 'data' is not defined

In [28]:
sns.set_theme()
sns.set_style("whitegrid")
plt.rc('font', family='serif')
plt.rc('xtick', labelsize='small')
plt.rc('ytick', labelsize='small')

g = sns.relplot(data=dfd, x="train_exc_limit", y=metric, hue='arch_pret', kind="line", palette="bright", col="mode", row="dataset", 
            height=5, aspect=1.5, legend=True, facet_kws={"despine": False})

g.set_ylabels("Test Accuracy Change (Abs %)")
g.set_xlabels("Samples per class");

g.savefig("figures/pret_change.pdf", bbox_inches='tight')


NameError: name 'sns' is not defined

# DF vs GF figure

In [29]:
from s3ts.data.acquisition import download_dataset
from s3ts.data.setup import train_test_splits
from s3ts.data.setup import setup_train_dm 
from pathlib import Path

# dataset settings
dataset         = "GunPoint"
storage_dir     = Path("storage")
cv_rep          = 0
exc             = 32
random_state    = 0
rho_dfs         = 0.1

X, Y, medoids, medoid_idx = download_dataset(dataset=dataset, storage_dir=storage_dir)
for j, (train_idx, test_idx) in enumerate(train_test_splits(X, Y, exc=exc, nreps=cv_rep+1, random_state=random_state)):
            if j == cv_rep:
                break

# sts settings
train_strat_size = 2
train_event_mult = 4
test_sts_length = 200
train_event_limit = exc
batch_size = 128

dm_df = setup_train_dm(X=X, Y=Y, patterns=medoids, train_idx=train_idx, test_idx=test_idx,
    test_sts_length=200, train_event_mult=4, train_strat_size=2, train_event_limit=exc,
    batch_size=128, val_size=0.25, rho_dfs=rho_dfs, window_length=10, mode="df", num_workers=4,
    window_time_stride=7, window_patt_stride=1,stride_series=False, random_state=random_state)

dm_gf = setup_train_dm(X=X, Y=Y, patterns=medoids, train_idx=train_idx, test_idx=test_idx,
    test_sts_length=200, train_event_mult=4, train_strat_size=2, train_event_limit=exc,
    batch_size=128, val_size=0.25, rho_dfs=rho_dfs, window_length=10, mode="gf", num_workers=4,
    window_time_stride=7, window_patt_stride=1,stride_series=False, random_state=random_state)

In [30]:

# plot settings
shift, n_frames = 0, 6
vmin, vmax = -2.5, 2.5
print_lines = False
fontsize = 18
ts_color = "darkslategray"
cmaps = ["seismic", "seismic"]

plt.rc('font', family='serif')
plt.rc('xtick', labelsize='small')
plt.rc('ytick', labelsize='small')


n_patterns = dm_df.n_patterns
sts_range = (dm_df.l_events*shift, dm_df.l_events*shift + dm_df.l_events*n_frames)

patts = dm_df.patterns.numpy()
TS = dm_df.STS_train[sts_range[0]:sts_range[1]]
DF = dm_df.DM_train[:,:,sts_range[0]:sts_range[1]]
GF = dm_gf.DM_train[:,:,sts_range[0]:sts_range[1]]

fig = plt.figure(figsize=(12, 1.5+n_patterns*1.5), dpi=100, layout="constrained")
gs = fig.add_gridspec(nrows=n_patterns+1, ncols=3,
        hspace=0, height_ratios=None,
        wspace=0, width_ratios=[0.1, 0.45, 0.45])


vlines = np.where(np.mod(np.arange(sts_range[0], sts_range[1]), dm_df.l_events) == 0)[0]

corner = fig.add_subplot(gs[0,0])
corner.set_xticklabels([]), corner.set_yticklabels([])
corner.set_xticks([]), corner.set_yticks([])
corner.text(0.95, 0.5, "STS", fontsize=fontsize, rotation=90, ha="center", va="center")
corner.text(0.5, 0.05, "Patterns", fontsize=fontsize, ha="center", va="center")
corner.spines['top'].set_visible(False)
corner.spines['right'].set_visible(False)
corner.spines['bottom'].set_visible(False)
corner.spines['left'].set_visible(False)


sts_ax1 = fig.add_subplot(gs[0,1])
sts_ax1.plot(np.arange(sts_range[0], sts_range[1]), TS, color=ts_color)
sts_ax1.set_xlim(sts_range[0], sts_range[1]-1)
sts_ax1.set_xticklabels([]), sts_ax1.set_yticklabels([])
sts_ax1.set_xticks([]), sts_ax1.set_yticks([])
if print_lines:
        [sts_ax1.axvline(x + sts_range[0], color="dimgray", linewidth=0.5) for x in vlines]
sts_ax1.grid(True)


sts_ax2 = fig.add_subplot(gs[0,2])
sts_ax2.plot(np.arange(sts_range[0], sts_range[1]), TS, color=ts_color)
sts_ax2.set_xlim(sts_range[0], sts_range[1]-1)
sts_ax2.set_xticklabels([]), sts_ax2.set_yticklabels([])
sts_ax2.set_xticks([]), sts_ax2.set_yticks([])
if print_lines:
        [sts_ax2.axvline(x + sts_range[0], color="dimgray", linewidth=0.5) for x in vlines]
sts_ax1.grid(True)


for p in range(n_patterns):

        # patt plot
        patt_ax = fig.add_subplot(gs[p+1,0])
        patt_ax.plot(patts[p,::-1], np.arange(len(patts[0])), color=ts_color)
        patt_ax.set_yticklabels([]), patt_ax.set_xticklabels([])
        patt_ax.set_yticks([]), patt_ax.set_xticks([])
        patt_ax.invert_xaxis(), patt_ax.grid(True)

        # # image plot
        # im_ax = fig.add_subplot(gs[p+1,1])     
        # gf = GF[p,:,sts_range[0]:sts_range[1]-(sts_range[1]-sts_range[0])//2]   
        # df = DF[p,:,sts_range[1]-(sts_range[1]-sts_range[0])//2:sts_range[1]]
        # gf = 2*(gf-gf.mean())/gf.std()
        # df = 2*(df-df.mean())/df.std()
        # im = np.concatenate([gf, df], axis=1)
        # im_ax.imshow(im, aspect="auto", cmap="seismic", vmin=-2, vmax=2)
        # im_ax.set_yticklabels([]), im_ax.set_xticklabels([])
        # im_ax.set_xticks([]), im_ax.set_yticks([])
        
        #[im_ax.axvline(x, color="white") for x in vlines]

        # df plot
        df_ax = fig.add_subplot(gs[p+1,1])        
        df_im = DF[p,:,sts_range[0]:sts_range[1]]
        df_im = 2*(df_im-df_im.mean())/df_im.std()
        df_ax.imshow(df_im, aspect="auto", cmap=cmaps[0], vmin=vmin, vmax=vmax)
        df_ax.set_yticklabels([]), df_ax.set_xticklabels([])
        df_ax.set_xticks([]), df_ax.set_yticks([])
        #df_vlines = np.where(np.mod(np.arange(sts_range[0], sts_range[1]-(sts_range[1]-sts_range[0])//2), dm_df.l_events) == 0)[0]
        #[df_ax.axvline(x, color="dimgray") for x in df_vlines]

        # gf plot
        gf_ax = fig.add_subplot(gs[p+1,2])
        gf_im = GF[p,:,sts_range[0]:sts_range[1]]
        gf_im = 2*(gf_im-gf_im.mean())/gf_im.std()
        gf_ax.imshow(gf_im, aspect="auto", cmap=cmaps[1], vmin=vmin, vmax=vmax)
        gf_ax.set_yticklabels([]), gf_ax.set_xticklabels([])
        gf_ax.set_xticks([]), gf_ax.set_yticks([])
        #gf_vlines = np.where(np.mod(np.arange(sts_range[1]-(sts_range[1]-sts_range[0])//2, sts_range[1]), dm_df.l_events) == 0)[0]
        #[gf_ax.axvline(x, color="dimgray") for x in gf_vlines]

        patt_ax.set_ylabel("Class {}".format(p+1), fontsize=fontsize)

        if p == n_patterns-1:
                df_ax.set_xlabel("DF Representation", fontsize=fontsize)
                gf_ax.set_xlabel("GAF Representation", fontsize=fontsize)

plt.savefig(f"figures/df_gf_{dataset}.pdf", bbox_inches="tight")

NameError: name 'plt' is not defined